In [ ]:
from flask import Flask, render_template, request
import cv2  # Optional for image processing

# Replace with paths to your trained model (model.pb) and label map (label_map.pbtxt)
model_path = "path/to/frozen_inference_graph.pb"
label_map_path = "path/to/label_map.pbtxt"

# Load the model and label map
detector = cv2.dnn_DetectionModel(model_path)  # OpenCV wrapper for OD API model
detector.setInputParams(size=(300, 300), scale=1.0 / 255)  # Adjust input size based on your model
categories = []  # List to store category labels

with open(label_map_path, 'r') as f:
    for line in f:
        categories.append(line.strip())

app = Flask(__name__)

def detect_pets(image_file):
    # Read the uploaded image
    image = cv2.imdecode(np.fromfile(image_file, dtype=np.uint8), cv2.IMREAD_COLOR)

    # Preprocess the image (resize) - Adjust based on your model requirements
    image = cv2.resize(image, (300, 300))
    image_blob = cv2.dnn.blobFromImage(image)

    # Perform object detection with the model
    detector.setInput(image_blob)
    outputs = detector.forward()

    # Process detections
    boxes = []
    confidences = []
    class_ids = []
    for detection in outputs[0, 0, :, :]:
        confidence = detection[2]
        if confidence > 0.5:  # Adjust threshold as needed
            box = detection[3:7] * np.array([image.shape[0], image.shape[1], image.shape[0], image.shape[1]])
            class_id = int(detection[1])
            boxes.append(box.astype(int))
            confidences.append(confidence)
            class_ids.append(class_ids)

    # Prepare results for visualization
    results = []
    for i in range(len(boxes)):
        x_min, y_min, x_max, y_max = boxes[i]
        class_name = categories[class_ids[i]]
        confidence = confidences[i]
        results.append({"class": class_name, "score": confidence, "bbox": [x_min, y_min, x_max, y_max]})

    return results

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/upload", methods=["POST"])
def upload():
    if request.method == "POST":
        # Get uploaded image file
        image_file = request.files["image"]
        if image_file:
            # Detect pets in the image
            results = detect_pets(image_file)
            return render_template("result.html", image_path=image_file.filename, detections=results)
        else:
            return render_template("error.html", message="No image uploaded!")

if __name__ == "__main__":
    app.run(debug=True)
